# TET–CVTL Framework – Simulazione Braiding Anyonico e Vacuum Torque

Notebook interattivo per esplorare la convergenza al trifoglio Nash attractor, visualizzare traiettorie e calcolare τ_vac.

Autore: Simon Soliman (TET Collective) – Febbraio 2026

In [ ]:
%matplotlib notebook
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, FloatSlider

## Definizione ODE e trifoglio ideale

In [ ]:
def braiding_ode(t, y, golden_phi=1.618034, torque_amp=0.05):
    x, y, z = y
    dx = np.cos(t) + 4 * np.cos(2 * t)
    dy = -np.sin(t) + 4 * np.sin(2 * t)
    dz = -3 * np.cos(3 * t)
    torque_factor = torque_amp * np.sin(2 * np.pi * t / (golden_phi ** 2))
    dx += torque_factor * (y * z - 0.5 * x)
    dy += torque_factor * (z * x - 0.5 * y)
    dz += torque_factor * (x * y - 0.5 * z)
    speed = np.sqrt(dx**2 + dy**2 + dz**2) + 1e-8
    return [dx / speed * 2, dy / speed * 2, dz / speed * 2]

def plot_trefoil_ideal():
    t = np.linspace(0, 12*np.pi, 800)
    x = np.sin(t) + 2*np.sin(2*t)
    y = np.cos(t) - 2*np.cos(2*t)
    z = -np.sin(3*t)
    return x, y, z

## Simulazione interattiva

In [ ]:
@interact(
    t_max=FloatSlider(min=10, max=200, step=10, value=100, description='t max'),
    torque_amp=FloatSlider(min=0.0, max=0.2, step=0.01, value=0.05, description='Ampiezza torque'),
    rtol=FloatSlider(min=1e-10, max=1e-5, step=1e-9, value=1e-8, description='rtol')
)
def run_simulation(t_max=100, torque_amp=0.05, rtol=1e-8):
    sol = solve_ivp(
        lambda t, y: braiding_ode(t, y, torque_amp=torque_amp),
        [0, t_max], [0.5, 0.0, 0.2],
        method='RK45', rtol=rtol, atol=1e-10, dense_output=True
    )
    
    t_eval = np.linspace(0, t_max, 2000)
    traj = sol.sol(t_eval)
    
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.plot(traj[0], traj[1], traj[2], 'cyan', lw=2, label='Traiettoria simulata')
    x,y,z = plot_trefoil_ideal()
    ax.plot(x, y, z, 'gold', '--', lw=2, label='Trifoglio ideale')
    
    ax.set_title(f'Simulazione TET–CVTL – t_max = {t_max}, torque_amp = {torque_amp}')
    ax.legend()
    plt.show()

## Calcolo τ_vac proxy

In [ ]:
def compute_tau_proxy(traj, t_eval):
    r = traj.T
    v = np.gradient(traj, t_eval, axis=1).T
    torque = np.cross(r, v, axis=1)
    return np.mean(np.linalg.norm(torque, axis=1)) * 1e-3

print("Esegui prima la cella di simulazione per calcolare τ_vac proxy")